In [50]:
#Connect drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import pandas as pd
import os

ROOT_PATH= "/content/drive/MyDrive/School"
DATA_PATH = os.path.join(ROOT_PATH,"AirQuality/raw_data/")

In [62]:
air_quality_23 = pd.read_csv(DATA_PATH + 'air_quality_city_2023.csv')
air_quality_24 = pd.read_csv(DATA_PATH + 'air_quality_city_2024.csv')
air_quality_25 = pd.read_csv(DATA_PATH + 'air_quality_city_2025.csv')
print("Shape of air_quality_23:", air_quality_23.shape)
print("Shape of air_quality_24:", air_quality_24.shape)
print("Shape of air_quality_25:", air_quality_25.shape)

Shape of air_quality_23: (339360, 14)
Shape of air_quality_24: (340320, 14)
Shape of air_quality_25: (254835, 14)


In [63]:
weather_23 = pd.read_csv(DATA_PATH + 'weather_city_2023.csv')
weather_24 = pd.read_csv(DATA_PATH + 'weather_city_2024.csv')
weather_25 = pd.read_csv(DATA_PATH + 'weather_city_2025.csv')
print("Shape of weather_23:", weather_23.shape)
print("Shape of weather_24:", weather_24.shape)
print("Shape of weather_25:", weather_25.shape)

Shape of weather_23: (338880, 32)
Shape of weather_24: (339840, 32)
Shape of weather_25: (256944, 32)


In [65]:
air_quality_missing_25 = pd.read_csv(DATA_PATH + 'air_quality_city_missing_2025.csv')
weather_missing_25 = pd.read_csv(DATA_PATH + 'weather_city_missing_2025.csv')
air_quality_missing_24 = pd.read_csv(DATA_PATH + 'air_quality_city_missing_2024.csv')
weather_missing_24 = pd.read_csv(DATA_PATH + 'weather_city_missing_2024.csv')
air_quality_missing_23 = pd.read_csv(DATA_PATH + 'air_quality_city_missing_2023.csv')
weather_missing_23 = pd.read_csv(DATA_PATH + 'weather_city_missing_2023.csv')

In [7]:
xa_phuong_city_name_TPHCM=pd.read_csv(DATA_PATH + 'xa_phuong_city_name_TPHCM.csv')
xa_phuong_city_name_TPHCM.head()

,city_name,district,latitude,longitude
0,Quận Một,Phường Sài Gòn,10.779197,106.699928
1,Quận Một,Phường Tân Định,10.790563,106.694460
2,Quận Ba,Phường Bến Thành,10.770381,106.695107
3,Quận Ba,Phường Cầu Ông Lãnh,10.762975,106.691090
4,Quận Ba,Phường Bàn Cờ,10.771177,106.683061


#Preprocess column

In [66]:
air_quatily_23_full = pd.concat([air_quality_23,air_quality_missing_23],ignore_index=True).drop_duplicates()
air_quality_24_full = pd.concat([air_quality_24,air_quality_missing_24],ignore_index=True).drop_duplicates()
air_quality_25_full = pd.concat([air_quality_25[pd.to_datetime(air_quality_25['timestamp_local']).dt.month != 10],air_quality_missing_25],ignore_index=True).drop_duplicates()
print("Shape of air_quality_23:", air_quatily_23_full.shape)
print("Shape of air_quality_24:", air_quality_24_full.shape)
print("Shape of air_quality_25:", air_quality_25_full.shape)

Shape of air_quality_23: (342252, 14)
Shape of air_quality_24: (343212, 14)
Shape of air_quality_25: (255969, 14)


In [67]:
weather_23_full = pd.concat([weather_23,weather_missing_23],ignore_index=True).drop_duplicates()
weather_24_full = pd.concat([weather_24,weather_missing_24],ignore_index=True).drop_duplicates()
weather_25_full = pd.concat([weather_25[pd.to_datetime(weather_25['timestamp_local']).dt.month != 10],weather_missing_25],ignore_index=True).drop_duplicates()
print(len(weather_23_full))
print(len(weather_24_full))
print(len(weather_25_full))

342252
343212
255969


In [68]:
def preprocess_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.title().str.replace(' ', '_')
    df=df.rename(columns={
        'City_Name':'Train Name',
        'district':'District',
        'latitude':"Latitude",
        'longitude':"Longitude"
    })
    return df

def preprocess_weather(df_weather):
    # Drop column không cần thiết
    df_weather = df_weather.drop(columns=[ 'App_Temp',"Revision_Version","Slp","Ts","Datetime",
                                          'Azimuth','Dewpt','Dhi','Dni','Elev_Angle','Ghi','H_Angle','Pod','Revision_Status','Snow','Solar_Rad','Vis','Wind_Dir','Wind_Gust_Spd','Weather','Start_Date','End_Date'])

    #Rename column
    df_weather = df_weather.rename(columns={
        'Timestamp_Local':'Local Time',
        'Clouds': 'Clouds',
        'Precip': 'Precipitation',
        'Pres': 'Pressure',
        'Rh': 'Relative Humidity',
        'Temp': 'Temperature',
        'Timestamp_Utc': 'UTC Time',
        'Uv': 'UV_Index',
        'Wind_Spd': 'Wind Speed'
    })
    df_weather = df_weather.sort_values(by='Local Time')
    df_weather['Local Time'] = pd.to_datetime(df_weather['Local Time'])
    df_weather.set_index('Local Time', inplace=True)
    return df_weather

def preprocess_air_quality(df_air_quality):
    df_air_quality=df_air_quality.drop(columns=[
        "Start_Date","End_Date","Ts","Datetime"
    ])
    df_air_quality=df_air_quality.rename(columns={
        'Timestamp_Local':'Local Time',
        'Timestamp_Utc':'UTC Time',
    })
    df_air_quality = df_air_quality.sort_values(by='Local Time')
    df_air_quality['Local Time'] = pd.to_datetime(df_air_quality['Local Time'])
    df_air_quality.set_index('Local Time', inplace=True)
    return df_air_quality

def preprocess(df,type):
  df = preprocess_column_names(df)
  if type == "weather":
    df = preprocess_weather(df)
  elif type == "air_quality":
    df = preprocess_air_quality(df)
  return df

In [69]:
air_quality_23_preprocess = preprocess(air_quatily_23_full,type="air_quality")
weather_23_preprocess = preprocess(weather_23_full,type="weather")


In [39]:
display(air_quality_23_preprocess)
display(weather_23_preprocess)

,Aqi,Co,No2,O3,Pm10,Pm25,So2,UTC Time,Train Name
Local Time,,,,,,,,,
2023-01-01,53,132.7,2.7,81.3,20.0,13.33,4.0,2022-12-31T17:00:00,Trảng Bàng
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Quận Ba
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Nhà Bè
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Ho Chi Minh City
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Quận Hai
...,...,...,...,...,...,...,...,...,...
2023-12-31,72,160.9,11.0,12.7,32.7,22.33,18.3,2023-12-30T17:00:00,Tân Phú
2023-12-31,43,40.7,3.0,70.3,19.3,10.33,5.0,2023-12-30T17:00:00,Thị Trấn Ngải Giao
2023-12-31,95,163.4,12.7,39.3,49.0,33.00,37.0,2023-12-30T17:00:00,Thủ Đức


,Clouds,Precipitation,Pressure,Relative Humidity,Temperature,UTC Time,UV_Index,Wind Speed,Train Name
Local Time,,,,,,,,,
2023-01-01 07:00:00,87,0.25,1013,83,23.0,2023-01-01T00:00:00,0.7,1.5,Quận Một
2023-01-01 07:00:00,87,0.25,1013,83,23.0,2023-01-01T00:00:00,0.7,1.5,Quận Chín
2023-01-01 07:00:00,87,0.25,1013,83,23.0,2023-01-01T00:00:00,0.7,1.5,Quận Bốn
2023-01-01 07:00:00,100,0.00,1001,89,23.0,2023-01-01T00:00:00,0.7,0.8,Thị Trấn Ngải Giao
2023-01-01 07:00:00,87,0.25,1011,83,23.0,2023-01-01T00:00:00,0.7,1.5,Thủ Đức
...,...,...,...,...,...,...,...,...,...
2023-12-31 06:00:00,68,0.00,1011,89,25.2,2023-12-30T23:00:00,0.0,2.8,Vũng Tàu
2023-12-31 06:00:00,100,0.00,1007,94,25.2,2023-12-30T23:00:00,0.0,0.8,Uyen Hung
2023-12-31 06:00:00,100,0.00,1011,94,25.2,2023-12-30T23:00:00,0.0,0.4,Củ Chi


In [41]:
air_quality_24_preprocess = preprocess(air_quality_24_full,type="air_quality")
weather_24_preprocess = preprocess(weather_24_full,type="weather")


In [42]:
display(air_quality_23_preprocess)
display(weather_23_preprocess)

,Aqi,Co,No2,O3,Pm10,Pm25,So2,UTC Time,Train Name
Local Time,,,,,,,,,
2023-01-01,53,132.7,2.7,81.3,20.0,13.33,4.0,2022-12-31T17:00:00,Trảng Bàng
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Quận Ba
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Nhà Bè
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Ho Chi Minh City
2023-01-01,169,320.8,79.3,24.3,100.7,69.33,141.0,2022-12-31T17:00:00,Quận Hai
...,...,...,...,...,...,...,...,...,...
2023-12-31,72,160.9,11.0,12.7,32.7,22.33,18.3,2023-12-30T17:00:00,Tân Phú
2023-12-31,43,40.7,3.0,70.3,19.3,10.33,5.0,2023-12-30T17:00:00,Thị Trấn Ngải Giao
2023-12-31,95,163.4,12.7,39.3,49.0,33.00,37.0,2023-12-30T17:00:00,Thủ Đức


,Clouds,Precipitation,Pressure,Relative Humidity,Temperature,UTC Time,UV_Index,Wind Speed,Train Name
Local Time,,,,,,,,,
2023-01-01 07:00:00,87,0.25,1013,83,23.0,2023-01-01T00:00:00,0.7,1.5,Quận Một
2023-01-01 07:00:00,87,0.25,1013,83,23.0,2023-01-01T00:00:00,0.7,1.5,Quận Chín
2023-01-01 07:00:00,87,0.25,1013,83,23.0,2023-01-01T00:00:00,0.7,1.5,Quận Bốn
2023-01-01 07:00:00,100,0.00,1001,89,23.0,2023-01-01T00:00:00,0.7,0.8,Thị Trấn Ngải Giao
2023-01-01 07:00:00,87,0.25,1011,83,23.0,2023-01-01T00:00:00,0.7,1.5,Thủ Đức
...,...,...,...,...,...,...,...,...,...
2023-12-31 06:00:00,68,0.00,1011,89,25.2,2023-12-30T23:00:00,0.0,2.8,Vũng Tàu
2023-12-31 06:00:00,100,0.00,1007,94,25.2,2023-12-30T23:00:00,0.0,0.8,Uyen Hung
2023-12-31 06:00:00,100,0.00,1011,94,25.2,2023-12-30T23:00:00,0.0,0.4,Củ Chi


In [43]:
air_quality_25_preprocess = preprocess(air_quality_25_full,type="air_quality")
weather_25_preprocess = preprocess(weather_25_full,type="weather")

In [44]:
display(air_quality_25_preprocess)
display(weather_25_preprocess)

,Aqi,Co,No2,O3,Pm10,Pm25,So2,UTC Time,Train Name
Local Time,,,,,,,,,
2025-01-01 00:00:00,64,152.9,17.0,64.7,21.3,18.33,36.0,2024-12-31T17:00:00,Quận Hai
2025-01-01 00:00:00,64,152.9,17.0,64.7,21.3,18.33,36.0,2024-12-31T17:00:00,Quận Mười
2025-01-01 00:00:00,32,97.9,4.3,60.7,7.7,7.67,12.3,2024-12-31T17:00:00,Dầu Tiếng
2025-01-01 00:00:00,54,222.5,53.0,17.3,16.7,14.00,61.7,2024-12-31T17:00:00,Tân Túc
2025-01-01 00:00:00,54,222.5,53.0,17.3,16.7,14.00,61.7,2024-12-31T17:00:00,Hóc Môn
...,...,...,...,...,...,...,...,...,...
2025-09-30 06:00:00,51,83.5,10.3,39.0,56.0,6.00,16.0,2025-09-29T23:00:00,Dĩ An
2025-09-30 06:00:00,51,59.7,3.0,44.7,56.0,6.00,7.3,2025-09-29T23:00:00,Chơn Thành
2025-09-30 06:00:00,51,83.5,10.3,39.0,56.0,6.00,16.0,2025-09-29T23:00:00,Thủ Dầu Một


,Clouds,Precipitation,Pressure,Relative Humidity,Temperature,UTC Time,UV_Index,Wind Speed,Train Name
Local Time,,,,,,,,,
2025-01-01 00:00:00,100.0,0.00,993.0,94.0,24.1,2024-12-31T17:00:00,0.0,1.20,Gia Ray
2025-01-01 00:00:00,100.0,0.00,1011.0,94.0,25.6,2024-12-31T17:00:00,0.0,2.00,Thành Phố Bà Rịa
2025-01-01 00:00:00,50.0,0.00,1011.0,69.0,26.0,2024-12-31T17:00:00,0.0,3.10,Quận Tân Phú
2025-01-01 00:00:00,100.0,0.00,1008.0,94.0,25.6,2024-12-31T17:00:00,0.0,2.00,Phuoc Tho
2025-01-01 00:00:00,50.0,0.00,1011.0,69.0,26.0,2024-12-31T17:00:00,0.0,3.10,Quận Sáu
...,...,...,...,...,...,...,...,...,...
2025-09-30 06:00:00,25.0,0.25,1007.0,100.0,25.0,2025-09-29T23:00:00,0.6,1.50,Quận Bốn
2025-09-30 06:00:00,34.0,1.00,1010.0,93.0,25.7,2025-09-29T23:00:00,0.6,1.36,Quận Bảy
2025-09-30 06:00:00,25.0,0.25,1007.0,100.0,25.0,2025-09-29T23:00:00,0.6,1.50,Quận Chín


In [46]:
xa_phuong_city_name_TPHCM_preprocess = preprocess_column_names(xa_phuong_city_name_TPHCM)

In [47]:
xa_phuong_city_name_TPHCM_preprocess.head()

,Train Name,District,Latitude,Longitude
0,Quận Một,Phường Sài Gòn,10.779197,106.699928
1,Quận Một,Phường Tân Định,10.790563,106.694460
2,Quận Ba,Phường Bến Thành,10.770381,106.695107
3,Quận Ba,Phường Cầu Ông Lãnh,10.762975,106.691090
4,Quận Ba,Phường Bàn Cờ,10.771177,106.683061


In [48]:
SAVE_PATH = os.path.join(ROOT_PATH,"AirQuality/preprocess_data/")
air_quality_23_preprocess.to_csv(SAVE_PATH + 'air_quality_23_preprocess.csv')
weather_23_preprocess.to_csv(SAVE_PATH + 'weather_23_preprocess.csv')
air_quality_24_preprocess.to_csv(SAVE_PATH + 'air_quality_24_preprocess.csv')
weather_24_preprocess.to_csv(SAVE_PATH + 'weather_24_preprocess.csv')
air_quality_25_preprocess.to_csv(SAVE_PATH + 'air_quality_25_preprocess.csv')
weather_25_preprocess.to_csv(SAVE_PATH + 'weather_25_preprocess.csv')

In [49]:
xa_phuong_city_name_TPHCM_preprocess.to_csv(SAVE_PATH + 'xa_phuong_city_name_TPHCM_preprocess.csv')

In [71]:
SAVE_PATH = os.path.join(ROOT_PATH,"AirQuality/preprocess_data/")
weather_23_preprocess.to_csv(SAVE_PATH + 'weather_23_preprocess.csv')